In [1]:
import nltk
from nltk.corpus import *
from nltk.stem.porter import *
import os
from Constants import *
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize
from pathlib import Path
from Trie import *

In [2]:
root = Path(".")
my_path = root / "Pickled_files" / "Documents"
dbfile = open(my_path, 'rb')     
docs = pickle.load(dbfile)
dbfile.close()

In [3]:
stop_words = set(stopwords.words('english'))

In [4]:
def isnumber(token):  
    for char in token:
        if not (char >= '0' and char <= '9'):
            return False
    return True

In [5]:
Prefix = Trie()
Suffix = Trie()
for sentence in docs: 
    sentence = sentence[0]
    tokens = nltk.tokenize.word_tokenize(str(sentence))
    new_token = list()
    for i in tokens:
        new_token.append(i.lower())
    tokens = new_token
    for words in new_token: 
        if words in stop_words or len(words) <= 2 or isnumber(words): 
            continue
        Prefix.insert(words)
        Suffix.insert(words[::-1])